In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras import layers

2023-05-10 15:38:57.048304: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-10 15:38:57.076800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [45]:
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [46]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


In [47]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [48]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28, 1)
(60000, 10)


In [49]:
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation="softmax"),
    ]
)

model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_7 (Dropout)         (None, 1600)            

In [50]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 3s 5ms/step - loss: 0.3755 - accuracy: 0.8841 - val_loss: 0.0845 - val_accuracy: 0.9773
Epoch 2/15
422/422 [==============================] - 2s 4ms/step - loss: 0.1175 - accuracy: 0.9640 - val_loss: 0.0570 - val_accuracy: 0.9855
Epoch 3/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0894 - accuracy: 0.9734 - val_loss: 0.0505 - val_accuracy: 0.9870
Epoch 4/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0738 - accuracy: 0.9771 - val_loss: 0.0441 - val_accuracy: 0.9870
Epoch 5/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0658 - accuracy: 0.9798 - val_loss: 0.0382 - val_accuracy: 0.9902
Epoch 6/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0575 - accuracy: 0.9814 - val_loss: 0.0393 - val_accuracy: 0.9900
Epoch 7/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0545 - accuracy: 0.9829 - val_loss: 0.0371 - val_accuracy: 0.9885
Epoch 

In [51]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.027312640100717545
Test accuracy: 0.9909999966621399


### Below is a bit about getting the shape of a dataset right:

In [5]:
embed_train = np.random.rand(9912, 4096)
train_labels = np.random.randint(0, 12, [9912, 12])
print(embed_train.shape)
print(train_labels.shape)

(9912, 4096)
(9912, 12)


In [8]:
model = keras.Sequential()

model.add(layers.Input(shape=(4096,)))
model.add(layers.Dense(units=2048, name='dense_1', activation='sigmoid', input_shape=(4096, 1)))
model.add(layers.Dense(units=2048, name='dense_2', activation='sigmoid'))
model.add(layers.Dense(units=12, name='output', activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='acc')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 2048)              8390656   
                                                                 
 dense_2 (Dense)             (None, 2048)              4196352   
                                                                 
 output (Dense)              (None, 12)                24588     
                                                                 
Total params: 12,611,596
Trainable params: 12,611,596
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(embed_train, train_labels, epochs=10)

Epoch 1/10
310/310 [==============================] - 1s 4ms/step - loss: -2667.2034 - acc: 0.0662
Epoch 2/10
310/310 [==============================] - 1s 3ms/step - loss: -4522.1392 - acc: 0.0974
Epoch 3/10
310/310 [==============================] - 1s 3ms/step - loss: -6450.4736 - acc: 0.0733
Epoch 4/10
310/310 [==============================] - 1s 3ms/step - loss: -8431.5117 - acc: 0.0964
Epoch 5/10
310/310 [==============================] - 1s 3ms/step - loss: -10448.8555 - acc: 0.0973
Epoch 6/10
310/310 [==============================] - 1s 3ms/step - loss: -12526.1758 - acc: 0.0952
Epoch 7/10
310/310 [==============================] - 1s 3ms/step - loss: -14675.5078 - acc: 0.1128
Epoch 8/10
310/310 [==============================] - 1s 3ms/step - loss: -16864.9648 - acc: 0.1036
Epoch 9/10
310/310 [==============================] - 1s 3ms/step - loss: -19075.9902 - acc: 0.1071
Epoch 10/10
310/310 [==============================] - 1s 3ms/step - loss: -21332.5684 - acc: 0.1052
